In [1]:
import polars as pl
import pandas as pd
import numpy as np
import pyarrow
import time

https://itnext.io/the-fastest-way-to-read-a-csv-file-in-pandas-2-0-532c1f978201

In [2]:
dataset_name = 'Kenya_90k_Set_1_w90_pW'
dataset_folder = "_".join(dataset_name.split('_')[:-2]) 
dataset_location = f'../Data_Storage_Processing/Data/{dataset_folder}/{dataset_name}.csv'

In [10]:
start_time = time.time()
df = pd.read_csv(dataset_location)
print(f'Pandas (Normal) Time: {np.round(time.time() - start_time,2)}[s]')  

Pandas (Normal) Time: 15.39[s]


In [11]:
start_time = time.time()
df = pd.read_csv(dataset_location, usecols=['short_ID','window_ID','window_start_date'])
print(f'Pandas Speciefed Cols Time: {np.round(time.time() - start_time,2)}[s]')  

Pandas Speciefed Cols Time: 6.98[s]


In [19]:
start_time = time.time()
df = pl.read_csv(dataset_location).to_pandas()
print(f'Polars Time: {np.round(time.time() - start_time,2)}[s]')  

Polars Time: 0.99[s]


In [6]:
df.head()

,short_ID,window_ID,window_start_date,d1,d2,d3,d4,d5,d6,d7,...,d81,d82,d83,d84,d85,d86,d87,d88,d89,d90
0,127,0,2018-01-20,0.543805,0.503693,0.463580,0.423467,0.383355,0.343242,0.303129,...,0.906409,0.904819,0.864707,0.786444,0.784481,0.706456,0.666459,0.626384,0.586360,0.583918
1,127,1,2018-04-20,0.525359,0.522565,0.480950,0.439334,0.397718,0.317612,0.314487,...,1.019053,0.977438,0.935822,0.894206,0.852591,0.810975,0.769359,0.727744,0.686128,0.644512
2,127,2,2018-07-19,0.651208,0.606258,0.561307,0.516357,0.471407,0.426456,0.381506,...,1.013146,0.968196,0.923246,0.878295,0.833345,0.788395,0.743444,0.698494,0.653543,0.608593
3,127,3,2018-10-17,0.594276,0.546883,0.499489,0.452096,0.404703,0.357309,0.309916,...,0.499489,0.452096,0.404703,0.357309,0.309916,0.262522,0.215129,0.167736,0.120342,0.072949
4,127,4,2019-01-15,0.953922,0.853922,0.753922,0.653922,0.553922,0.453922,0.353922,...,0.453922,0.353922,0.253922,0.153922,0.753922,0.653922,0.553922,0.453922,0.353922,2.000000


In [20]:
start_time = time.time()
df = pl.read_csv(dataset_location, columns=['short_ID','window_ID','window_start_date']).to_pandas()
print(f'Polars Speciefed Cols Time: {np.round(time.time() - start_time,2)}[s]')  

Polars Speciefed Cols Time: 0.6[s]


In [18]:
start_time = time.time()
df = pl.read_csv(dataset_location, use_pyarrow=True).to_pandas()
print(f'Polars Pyarrow Time: {np.round(time.time() - start_time,2)}[s]')  

Polars Pyarrow Time: 3.38[s]


In [30]:
start_time = time.time()
row = (pl.scan_csv(dataset_location)  # lazy, doesn't do a thing
    # .select(
    #     ["short_ID", "window_ID"]
    # )  # select only 2 columns (other columns will not be read)
    .filter(
        (pl.col("short_ID") == 127) & (pl.col("window_ID") == 0)
    )  # the filter is pushed down the scan, so less data is read into memory
    .fetch(100)  # pushed a limit of 100 rows to the scan level
).to_pandas()
print(f'Polars Scan Row Time: {np.round(time.time() - start_time,2)}[s]')  

Polars Scan Row Time: 0.01[s]


In [31]:
row

,short_ID,window_ID,window_start_date,d1,d2,d3,d4,d5,d6,d7,...,d81,d82,d83,d84,d85,d86,d87,d88,d89,d90
0,127,0,2018-01-20,0.543805,0.503693,0.46358,0.423467,0.383355,0.343242,0.303129,...,0.906409,0.904819,0.864707,0.786444,0.784481,0.706456,0.666459,0.626384,0.58636,0.583918


> ***Observations:***  
> - Polars is much faster (x10s)
> - Specifying the columns saves a lot of time
> - Polars Scan CSV allow to pool specific row X100 faster